In [27]:
from napari_stress._reconstruction.fit_utils import _fibonacci_sampling
from napari_stress._reconstruction.patches import _find_neighbor_indices, _orient_patch, _calculate_mean_curvature_on_patch, _fit_quadratic_surface, _estimate_patch_radii
import numpy as np

# first create fibonacci pointcloud
pointcloud = _fibonacci_sampling(number_of_points=256)

# assert that distance of every point to origin is 1
assert np.allclose(np.linalg.norm(pointcloud, axis=1), 1)

# make sure every point has one neighbor (itself)
patch_indices = _find_neighbor_indices(pointcloud, patch_radius=0.1)
assert np.allclose(np.asarray(patch_indices).squeeze(), np.arange(256))

# make sure all points are neighbors of each other for large radius
patch_indices = _find_neighbor_indices(pointcloud, patch_radius=2)
assert np.allclose(np.asarray([len(x) for x in patch_indices]), np.repeat(256, 256))

# extract a patch around the first point
# assert that all of these points have a distance of less than radius to the first point
radius = 0.3
patch_indices = _find_neighbor_indices(pointcloud, patch_radius=radius)
assert all(np.linalg.norm(pointcloud[patch_indices[0]] - pointcloud[0][None, :], axis=1) < radius)

# get patch and orient it
patch = pointcloud[patch_indices[0]]
transformed_patch, _, orient_matrix = _orient_patch(patch, patch[0])
assert np.array_equal(transformed_patch.shape, patch.shape)
assert np.allclose(transformed_patch.mean(axis=0), 0)  # patch center should be zero

# make sure it's aligned with the first axis
_, _, _orient_matrix = _orient_patch(transformed_patch, [0,0,0])
assert np.allclose(_orient_matrix[:, 0], [1, 0, 0])

# calculate principal curvatures on patch
fit_params = _fit_quadratic_surface(transformed_patch)
curvature, principal_curvatures = _calculate_mean_curvature_on_patch(
    transformed_patch[0], fit_params)

assert abs(curvature[0] - 1) < 0.1  # curvature error should be smaller than 10%

# calculate all principal curvatures
k1 = []
k2 = []
for i, p in enumerate(pointcloud):
    patch = pointcloud[patch_indices[i]]
    transformed_patch, _, orient_matrix = _orient_patch(patch, patch[0])
    fit_params = _fit_quadratic_surface(transformed_patch)
    curvature, principal_curvatures = _calculate_mean_curvature_on_patch(
        transformed_patch[0], fit_params)
    k1.append(principal_curvatures[0][0])
    k2.append(principal_curvatures[0][1])

In [29]:
_estimate_patch_radii(pointcloud, patch_indices, k1, k2, 0.1, 0.1, 0.1, 0.1)

[1.0107186827961825,
 -1.0167611637950666,
 -1.0609319325108908,
 1.0089333188355254,
 1.0152614890216376,
 1.0155132174342547,
 1.0106302122966555,
 1.0140730674475078,
 1.0130514089971203,
 -1.0143653612584587,
 1.0128271711944556,
 -1.0154963702526767,
 1.0098372445012247,
 1.0203370644225485,
 -1.0213471851314142,
 1.020468950543935,
 -1.0205154856383798,
 1.0111855818585103,
 -1.0186985745910828,
 -1.0182344479108336,
 1.0159754420959826,
 1.0108747043730635,
 -1.0156101106317446,
 -1.0153351540903444,
 -1.0151422790731375,
 -1.015033135751912,
 -1.0150075842225672,
 -1.015059458507746,
 1.011738321133906,
 -1.0153425016400452,
 -1.0155444255217012,
 1.011246939135789,
 -1.0160149325669487,
 1.0108039504689879,
 -1.0194825010004396,
 -1.0191778615072153,
 -1.01897846802413,
 -1.0188732866756673,
 -1.0188491474417454,
 -1.0188922429809337,
 1.0127328255569337,
 1.012293143982215,
 -1.019301689042356,
 1.0113652624379066,
 -1.0197153047155976,
 -1.0199453663518212,
 1.00991467549294